In [2]:
!pip install opencv-python
!pip install numpy pandas


Unable to create process using 'C:\Users\Sincerely Stepper\anaconda3\python.exe "C:\Users\Sincerely Stepper\anaconda3\Scripts\pip-script.py" install opencv-python'
Unable to create process using 'C:\Users\Sincerely Stepper\anaconda3\python.exe "C:\Users\Sincerely Stepper\anaconda3\Scripts\pip-script.py" install numpy pandas'


In [ ]:


import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------- CONFIG ------------------
CSV_INPUT_IMAGE = 'input_image.csv'
FPGA_SOBEL_OUT = 'vivado_sobel_output.csv'
FPGA_CANNY_OUT = 'vivado_canny_output.csv'

SAVE_GOLDEN = True  # Save golden outputs to CSVs
LOW_THRESH = 50
HIGH_THRESH = 150
TOLERANCE = 0        # Tolerance for pixel mismatch
# ----------------------------------------

# --- Load grayscale image from CSV ---
def load_image(csv_path):
    return pd.read_csv(csv_path, header=None).values.astype(np.uint8)

# --- Run Sobel filter using OpenCV ---
def sobel_reference(img):
    sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = cv2.convertScaleAbs(np.abs(sobel_x) + np.abs(sobel_y))
    return magnitude

# --- Run Canny edge detection using OpenCV ---
def canny_reference(img, low_thresh, high_thresh):
    return cv2.Canny(img, low_thresh, high_thresh)

# --- Compare Vivado output with reference ---
def compare_outputs(fpga_csv, golden_output, label="Sobel"):
    fpga_output = pd.read_csv(fpga_csv, header=None).values.astype(np.uint8)
    if golden_output.shape != fpga_output.shape:
        raise ValueError(f"Shape mismatch: FPGA {fpga_output.shape}, Golden {golden_output.shape}")

    diff = np.abs(fpga_output.astype(np.int32) - golden_output.astype(np.int32))
    mismatches = (diff > TOLERANCE).astype(np.uint8)
    accuracy = 100 * np.mean(mismatches == 0)

    print(f"[{label}] Accuracy: {accuracy:.2f}%")
    return accuracy, mismatches

# --- Save output image to CSV ---
def save_output(img_array, filename):
    pd.DataFrame(img_array).to_csv(filename, index=False, header=False)

# --- Show differences visually ---
def show_comparison(input_img, golden, fpga, mismatch_mask, label):
    plt.figure(figsize=(12, 5))
    plt.suptitle(f"{label} Comparison")

    plt.subplot(1, 4, 1)
    plt.title("Input")
    plt.imshow(input_img, cmap='gray')

    plt.subplot(1, 4, 2)
    plt.title("Golden")
    plt.imshow(golden, cmap='gray')

    plt.subplot(1, 4, 3)
    plt.title("FPGA Output")
    plt.imshow(fpga, cmap='gray')

    plt.subplot(1, 4, 4)
    plt.title("Mismatch (Red)")
    plt.imshow(mismatch_mask * 255, cmap='Reds')

    plt.tight_layout()
    plt.show()

# ========== MAIN PIPELINE ==========
input_img = load_image(CSV_INPUT_IMAGE)

# --- Golden references ---
sobel_ref = sobel_reference(input_img)
canny_ref = canny_reference(input_img, LOW_THRESH, HIGH_THRESH)

# --- Save golden outputs (optional) ---
if SAVE_GOLDEN:
    save_output(sobel_ref, "golden_sobel_output.csv")
    save_output(canny_ref, "golden_canny_output.csv")

# --- Compare Sobel ---
fpga_sobel = pd.read_csv(FPGA_SOBEL_OUT, header=None).values.astype(np.uint8)
sobel_acc, sobel_diff = compare_outputs(FPGA_SOBEL_OUT, sobel_ref, label="Sobel")

# --- Compare Canny ---
fpga_canny = pd.read_csv(FPGA_CANNY_OUT, header=None).values.astype(np.uint8)
canny_acc, canny_diff = compare_outputs(FPGA_CANNY_OUT, canny_ref, label="Canny")

# --- Visuals ---
show_comparison(input_img, sobel_ref, fpga_sobel, sobel_diff, label="Sobel")
show_comparison(input_img, canny_ref, fpga_canny, canny_diff, label="Canny")
